In [2]:
#  output.csv dataframe
import pandas as pd

df = pd.read_csv('output.csv')


In [2]:
df

,wsi_name,label,subtype,csv_dir
0,TCGA-W2-A7H5-01Z-00-DX1.99C575DE-69FA-4F0D-A8D...,PCPG,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
1,TCGA-QR-A6H1-01A-01-TS1.A73EB512-D30D-4AAB-8E0...,PCPG,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
2,TCGA-RW-A8AZ-01A-01-TSA.4032759C-A195-4E5F-AF4...,PCPG,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
3,TCGA-QR-A708-01Z-00-DX1.2090CC30-E47C-4A54-9A7...,PCPG,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
4,TCGA-TT-A6YO-01A-01-TS1.4C689C83-4D46-4EC0-8E6...,PCPG,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
...,...,...,...,...
1545,TCGA-OR-A5JM-01Z-00-DX5.4B3D3E4D-E577-41AF-823...,ACC,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
1546,TCGA-OR-A5LN-01Z-00-DX5.343B333F-068C-4AD4-9E5...,ACC,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
1547,TCGA-OR-A5LS-01Z-00-DX5.275F7E68-0372-408F-9BF...,ACC,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...
1548,TCGA-OR-A5LD-01A-01-TSA.C5103F20-E171-4670-B98...,ACC,Endocrine,/hpc2hdd/JH_DATA/share/ysi538/PrivateShareGrou...


In [3]:
#  df test_dataset subtype
import os
import shutil
import openslide
import random
import numpy as np
from tqdm import tqdm

for index, row in tqdm(df.iterrows()):
    if not os.path.exists('test_dataset/' + row['subtype']):
        os.makedirs('test_dataset/' + row['subtype'])
    # 
    wsi_path = row['csv_dir']
    #  OpenSlide wsi 
    #  level level 
 
    
    slide = openslide.OpenSlide(wsi_path)
    width, height = slide.dimensions
    slide_info = slide.properties
    level_count = slide.level_count
    for level in range(level_count):
        ratio = slide.level_downsamples[level]
        width = int(slide_info.get(f"openslide.level[{level}].width"))
        height = int(slide_info.get(f"openslide.level[{level}].height"))
        i = 0
        count = 0
        while i < 3:

            if os.path.exists('test_dataset/' + row['subtype'] + '/' + row['wsi_name'] + '_' + row['label'] + '_' + str(level) + '_' + str(i) + '.png'):
                i += 1
                continue
            if level == level_count - 1:
                if width < 448 or height < 448:
                    break
                x = random.randint(0, width - 448)
                y = random.randint(0, height - 448)
                region = slide.read_region((int(x * ratio), int(y * ratio)), level, (448, 448))
            else:
                if width < 1120 or height < 1120:
                    break
                x = random.randint(0, width - 1120)
                y = random.randint(0, height - 1120)
                region = slide.read_region((int(x * ratio), int(y * ratio)), level, (1120, 1120))

            #  
            region_np = np.array(region)
            white_ratio = np.sum(region_np >= 240) / region_np.size

            if white_ratio < 0.8:
                region.save('test_dataset/' + row['subtype'] + '/' + row['wsi_name'] + '_' + row['label'] + '_' + str(level) + '_' + str(i) + '.png')
                i += 1

            count += 1
            if count > 100:
                break
    slide.close()



1550it [2:21:18,  5.47s/it]
